**code after reducing** -> punctuation, numbers, stopwords, emojis


with English words

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re,json,nltk
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score,precision_score,recall_score,f1_score
#stopwords_list ='stopwords-bn.txt'


import re
from google.colab import files
import string

In [4]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import pandas as pd

In [5]:
data = pd.read_excel('/content/drive/MyDrive/4.2_resources/softcom/data/final_tech_comment_dataset_with_annotation.xlsx')

In [6]:
len(data)

3037

In [7]:
data['Sentiment'] = data['Person 3'].copy()

In [8]:
print("Total Reviews:",len(data),
      "\nTotal Positive Reviews:",len(data[data.Sentiment == 1]),
      "\nTotal Neutral Reviews:",len(data[data.Sentiment == 2]),
      "\nTotal Negative Reviews:",len(data[data.Sentiment == 0]))

Total Reviews: 3037 
Total Positive Reviews: 2255 
Total Neutral Reviews: 291 
Total Negative Reviews: 491


In [9]:
data.columns

Index(['Bengali tech product reviews', 'tech product', 'Person 1', 'Person 2',
       'Person 3', 'Sentiment'],
      dtype='object')

In [10]:
data.rename(columns={'Bengali tech product reviews': 'Comment'}, inplace=True)

In [11]:
# print some unprocessed reviews
sample_data = [573, 1209, 864, 297, 1502, 639, 2085, 431, 775, 1926, 1098, 256, 1823, 904, 117, 1985, 623, 1490, 765, 2217]
for i in sample_data:
      print(data.Comment[i],'\n','Sentiment:-- ',data.Sentiment[i],'\n')

জিনিস টা ভাল বোতাম ফোনে খুব ভাল কাজ করে। 
 Sentiment:--  1 

আমার খুব পছন্দ হয়েছে। তবে দামটা একটু বেশি হয়ে গিয়েছে। এর আগে দেখে ছিলাম তখন ৮০০/- টাকা ছিল। দাম কম রাখলে আরো বেশি সেল হবে। ধন্যবাদ দারাজ ও সেলার ভাই কে। 
 Sentiment:--  1 

গতকালই ক্যাশ অন ডেলিভারিতে কিনলাম। পাওয়ার অন-অফ এবং ভলিউম বাটনে সমস্যা। বেশি জোরে এবং বেশি সময় ধরে চাপ দিয়ে অন-অফ করতে হয়। আমি ত্রুটিযুক্ত এই ব্লুটুথ নেকব্যান্ডের পরিবর্তে অন্য আরেকটি নেকব্যান্ড চাই। অন্যথায় বাকি জীবনে কখনই আর দারাজ থেকে কোন পন্য কিনবো না আর কাউকে কেনার কথা বলবোও না। 
 Sentiment:--  0 

প্যাকেজিংটা অনেক ভালো ছিল ধন্যবাদ 
 Sentiment:--  1 

একটা কিনে একটি ফ্রি পেয়েছি । মোবাইলের স্ট্যান্ড টা খুব দারুন ছিল এবং হেডফোনটা অনেক ভালো. আমার কাছে খুব দারুন লেগেছে আপনারা চাইলে নিতে পারেন বেশি দামি না কিনে এটা কিনে দেখতে পারেন অনেক ভালো হবে🌟💥❤ধন্যবাদ দারাজ এবং সেলার 
 Sentiment:--  1 

সাউন্ড পটপট শব্দ করে।রিটার্ন করে দিছি। 
 Sentiment:--  0 

আলহামদুলিল্লাহ যেমনি চেয়েছি তেমনি পেয়েছি,
অনেক ভালো পণ হাতে পেয়েছি, 
 Sentiment:--  1 

আমি কখনও অবাস্তব অফার

In [12]:
data.columns

Index(['Comment', 'tech product', 'Person 1', 'Person 2', 'Person 3',
       'Sentiment'],
      dtype='object')

In [15]:
import re
import string

def clean_text(text):
    # Remove numbers
    text = re.sub(r'\d+', '', str(text))

    # Remove all punctuation using string.punctuation
    # review = re.sub('[^\u0980-\u09FF]',' ',str(review)) #removing unnecessary punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))  # Keeping Bengali characters and specified punctuation

    # Remove English words

    # Remove emojis
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    text = emoji_pattern.sub(r'', str(text))

    return text

In [16]:
# Apply the function to the dataframe

data['cleaned'] = data['Comment'].apply(clean_text)

# Print some cleaned reviews from the dataset
sample_data = [573, 1209, 864, 297, 1502, 639, 2085, 431, 775, 1926, 1098, 256, 1823, 904, 117, 1985, 623, 1490, 765, 2217]

for i in sample_data:
    print('Original:\n', data.Comment[i], '\nCleaned:\n', data.cleaned[i], '\n', 'Sentiment:-- ', data.Sentiment[i], '\n')


Original:
 জিনিস টা ভাল বোতাম ফোনে খুব ভাল কাজ করে। 
Cleaned:
 জিনিস টা ভাল বোতাম ফোনে খুব ভাল কাজ করে। 
 Sentiment:--  1 

Original:
 আমার খুব পছন্দ হয়েছে। তবে দামটা একটু বেশি হয়ে গিয়েছে। এর আগে দেখে ছিলাম তখন ৮০০/- টাকা ছিল। দাম কম রাখলে আরো বেশি সেল হবে। ধন্যবাদ দারাজ ও সেলার ভাই কে। 
Cleaned:
 আমার খুব পছন্দ হয়েছে। তবে দামটা একটু বেশি হয়ে গিয়েছে। এর আগে দেখে ছিলাম তখন  টাকা ছিল। দাম কম রাখলে আরো বেশি সেল হবে। ধন্যবাদ দারাজ ও সেলার ভাই কে। 
 Sentiment:--  1 

Original:
 গতকালই ক্যাশ অন ডেলিভারিতে কিনলাম। পাওয়ার অন-অফ এবং ভলিউম বাটনে সমস্যা। বেশি জোরে এবং বেশি সময় ধরে চাপ দিয়ে অন-অফ করতে হয়। আমি ত্রুটিযুক্ত এই ব্লুটুথ নেকব্যান্ডের পরিবর্তে অন্য আরেকটি নেকব্যান্ড চাই। অন্যথায় বাকি জীবনে কখনই আর দারাজ থেকে কোন পন্য কিনবো না আর কাউকে কেনার কথা বলবোও না। 
Cleaned:
 গতকালই ক্যাশ অন ডেলিভারিতে কিনলাম। পাওয়ার অনঅফ এবং ভলিউম বাটনে সমস্যা। বেশি জোরে এবং বেশি সময় ধরে চাপ দিয়ে অনঅফ করতে হয়। আমি ত্রুটিযুক্ত এই ব্লুটুথ নেকব্যান্ডের পরিবর্তে অন্য আরেকটি নেকব্যান্ড চাই। অন্যথায় বাকি জীবনে কখনই আর দ

In [17]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

# Load Bengali stopwords
stop_words = set(stopwords.words('bengali'))

# Function to remove stopwords from text
def remove_stopwords(text):
    words = text.split()  # Split the text into words
    filtered_words = [word for word in words if word.lower() not in stop_words]  # Remove stopwords
    return ' '.join(filtered_words)  # Recreate text without stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [18]:
print(stop_words)

{'আছে', 'বলতে', 'হয়েছিল', 'এখানেই', 'করতে', 'পেয়্র্', 'যত', 'শুরু', 'হওয়ার', 'মধ্যেই', 'তেমন', 'আগে', 'ফের', 'তাঁাহারা', 'যিনি', 'সুতরাং', 'করবে', 'যাচ্ছে', 'গিয়ে', 'কত', 'মধ্যেও', 'রয়েছে', 'ইহা', 'দেখে', 'ই', 'যতটা', 'ঠিক', 'নাগাদ', 'দ্বারা', 'ওখানে', 'এমনি', 'গেছে', 'আপনি', 'নয়', 'কয়েক', 'এমন', 'নতুন', 'তাহাতে', 'হতেই', 'কিছু', 'রকম', 'মাত্র', 'শুধু', 'অবশ্য', 'যাতে', 'ইত্যাদি', 'কিংবা', 'আদ্যভাগে', 'কারণ', 'আপনার', 'নিয়ে', 'করিতে', 'জন', 'আমাকে', 'ব্যাপারে', 'সামনে', 'তিনি', 'পারে', 'তাই', 'ধরে', 'কাজ', 'কাজে', 'দিতে', 'সে', 'ধরা', 'কোনও', 'ফিরে', 'দিন', 'যাওয়া', 'রেখে', 'থাকেন', 'প্রথম', 'প্রযন্ত', 'যাকে', 'পরেই', 'ওই', 'কেউ', 'নিজে', 'বন', 'দুটি', 'ভাবে', 'প্রায়', 'পরে', 'চালু', 'এঁদের', 'নানা', 'জন্যওজে', 'বি', 'বাদে', 'কয়েকটি', 'পর্যন্ত', 'মতোই', 'যাওয়ার', 'এখানে', 'ওর', 'এখনও', 'কী', 'তখন', 'সি', 'যেন', 'তাদের', 'এদের', 'জানতে', 'স্পষ্ট', 'করেন', 'সঙ্গে', 'জানানো', 'করি', 'এল', 'এবার', 'নিয়ে', 'পর', 'জানা', 'জানিয়েছে', 'মতো', 'হওয়ায়', 'হয়েছেন', 'র', 'সব', 'দিলেন', 

In [19]:
# Apply the function to the dataframe
data['cleaned_stopwords'] = data['cleaned'].apply(remove_stopwords)

# Print some cleaned reviews from the dataset
sample_data = [573, 1209, 864, 297, 1502, 639, 2085, 431, 775, 1926, 1098, 256, 1823, 904, 117, 1985, 623, 1490, 765, 2217]

for i in sample_data:
    print('cleaned:\n', data.cleaned[i], '\ncleaned_stopwords:\n', data.cleaned_stopwords[i], '\n', 'Sentiment:-- ', data.Sentiment[i], '\n')


cleaned:
 জিনিস টা ভাল বোতাম ফোনে খুব ভাল কাজ করে। 
cleaned_stopwords:
 জিনিস টা ভাল বোতাম ফোনে ভাল করে। 
 Sentiment:--  1 

cleaned:
 আমার খুব পছন্দ হয়েছে। তবে দামটা একটু বেশি হয়ে গিয়েছে। এর আগে দেখে ছিলাম তখন  টাকা ছিল। দাম কম রাখলে আরো বেশি সেল হবে। ধন্যবাদ দারাজ ও সেলার ভাই কে। 
cleaned_stopwords:
 পছন্দ হয়েছে। দামটা একটু হয়ে গিয়েছে। ছিলাম টাকা ছিল। দাম কম রাখলে আরো সেল হবে। ধন্যবাদ দারাজ সেলার ভাই কে। 
 Sentiment:--  1 

cleaned:
 গতকালই ক্যাশ অন ডেলিভারিতে কিনলাম। পাওয়ার অনঅফ এবং ভলিউম বাটনে সমস্যা। বেশি জোরে এবং বেশি সময় ধরে চাপ দিয়ে অনঅফ করতে হয়। আমি ত্রুটিযুক্ত এই ব্লুটুথ নেকব্যান্ডের পরিবর্তে অন্য আরেকটি নেকব্যান্ড চাই। অন্যথায় বাকি জীবনে কখনই আর দারাজ থেকে কোন পন্য কিনবো না আর কাউকে কেনার কথা বলবোও না। 
cleaned_stopwords:
 গতকালই ক্যাশ অন ডেলিভারিতে কিনলাম। পাওয়ার অনঅফ ভলিউম বাটনে সমস্যা। জোরে সময় চাপ দিয়ে অনঅফ হয়। ত্রুটিযুক্ত ব্লুটুথ নেকব্যান্ডের পরিবর্তে আরেকটি নেকব্যান্ড চাই। অন্যথায় বাকি জীবনে কখনই দারাজ পন্য কিনবো কেনার কথা বলবোও না। 
 Sentiment:--  0 

cleaned:
 প্যাকে

In [20]:
pip install transformers

In [21]:
import pandas as pd

# Assuming 'data' is your DataFrame
selected_columns = ['cleaned_stopwords', 'Sentiment']
selected_data = data[selected_columns]

# Save the selected columns to a CSV file
selected_data.to_csv('/content/drive/MyDrive/4.2_resources/softcom/data/project_selected_dataD.csv', index=False)

In [22]:
cleaned_data = pd.read_csv('/content/drive/MyDrive/4.2_resources/softcom/data/project_selected_dataD.csv')

In [23]:
print(cleaned_data.columns)
print("Total Reviews:",len(cleaned_data))

Index(['cleaned_stopwords', 'Sentiment'], dtype='object')
Total Reviews: 3037


In [24]:
cleaned_data.columns

Index(['cleaned_stopwords', 'Sentiment'], dtype='object')

In [25]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch

# Assuming 'cleaned_data' is your DataFrame
X = cleaned_data['cleaned_stopwords'].astype(str)  # Convert to string to handle potential non-string values
y = cleaned_data['Sentiment']

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertModel.from_pretrained('bert-base-cased')

batch_size = 4
tokenized_data = []

# Check for null values in 'cleaned_stopwords' column
null_values = X[X.isnull()]
if not null_values.empty:
    print("Warning: There are null values in 'cleaned_stopwords' column. Handle accordingly.")

for i in range(0, len(X), batch_size):
    batch_texts = X.iloc[i:i+batch_size].tolist()
    batch_encoding = tokenizer.batch_encode_plus(
        batch_texts,
        add_special_tokens=True,
        padding='max_length',
        truncation=True,
        max_length=62,
        return_tensors='pt'
    )
    tokenized_data.append(batch_encoding)

input_ids = torch.cat([d['input_ids'] for d in tokenized_data], dim=0)
attention_mask = torch.cat([d['attention_mask'] for d in tokenized_data], dim=0)

with torch.no_grad():
    model.eval()
    outputs = model(input_ids, attention_mask=attention_mask)

bert_embeddings = outputs.last_hidden_state[:, 0, :]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [26]:
len(bert_embeddings)

3037

In [27]:
import pickle

In [28]:
import pickle

save_dict = {
    'bert_embeddings': bert_embeddings,
    'y_encoded': y
}

with open('/content/drive/MyDrive/4.2_resources/softcom/data/bert_and_y_encodedD.pkl', 'wb') as file:
    pickle.dump(save_dict, file)


In [29]:
with open('/content/drive/MyDrive/4.2_resources/softcom/data/bert_and_y_encodedD.pkl', 'rb') as file:
    loaded_dict = pickle.load(file)

loaded_bert_embeddings = loaded_dict['bert_embeddings']
loaded_y_encoded = loaded_dict['y_encoded']


In [30]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report

def batch_train(model, X_train_batches, y_train_batches, X_test, y_test):
    for X_batch, y_batch in zip(X_train_batches, y_train_batches):
        model.fit(X_batch, y_batch)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    classification_rep = classification_report(y_test, y_pred)

    return accuracy, classification_rep

# Assuming loaded_bert_embeddings and loaded_y_encoded are defined
X_train, X_test, y_train, y_test = train_test_split(loaded_bert_embeddings, loaded_y_encoded, test_size=0.4, random_state=42)

batch_size = 50

X_train_batches = [X_train[i:i+batch_size] for i in range(0, len(X_train), batch_size)]
y_train_batches = [y_train[i:i+batch_size] for i in range(0, len(y_train), batch_size)]

# K-Nearest Neighbors (KNN) Classifier
knn_classifier = KNeighborsClassifier(n_neighbors=5)
knn_accuracy, knn_classification_rep = batch_train(knn_classifier, X_train_batches, y_train_batches, X_test, y_test)

print("K-Nearest Neighbors (KNN) Classifier:")
print(f"Accuracy: {knn_accuracy}")
print("Classification Report:\n", knn_classification_rep)

# Decision Tree (DT) Classifier
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_accuracy, dt_classification_rep = batch_train(dt_classifier, X_train_batches, y_train_batches, X_test, y_test)

print("\nDecision Tree (DT) Classifier:")
print(f"Accuracy: {dt_accuracy}")
print("Classification Report:\n", dt_classification_rep)

# Linear Support Vector Machine (SVM) Classifier
linear_svm_classifier = SVC(kernel='linear', C=1.0)
linear_svm_accuracy, linear_svm_classification_rep = batch_train(linear_svm_classifier, X_train_batches, y_train_batches, X_test, y_test)

print("\nLinear Support Vector Machine (SVM) Classifier:")
print(f"Accuracy: {linear_svm_accuracy}")
print("Classification Report:\n", linear_svm_classification_rep)

# Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_accuracy, rf_classification_rep = batch_train(rf_classifier, X_train_batches, y_train_batches, X_test, y_test)

print("\nRandom Forest Classifier:")
print(f"Accuracy: {rf_accuracy}")
print("Classification Report:\n", rf_classification_rep)

# Gradient Boosting Classifier
gb_classifier = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_accuracy, gb_classification_rep = batch_train(gb_classifier, X_train_batches, y_train_batches, X_test, y_test)

print("\nGradient Boosting Classifier:")
print(f"Accuracy: {gb_accuracy}")
print("Classification Report:\n", gb_classification_rep)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


K-Nearest Neighbors (KNN) Classifier:
Accuracy: 0.6724279835390946
Classification Report:
               precision    recall  f1-score   support

           0       0.19      0.17      0.18       195
           1       0.75      0.87      0.81       905
           2       0.00      0.00      0.00       115

    accuracy                           0.67      1215
   macro avg       0.32      0.35      0.33      1215
weighted avg       0.59      0.67      0.63      1215


Decision Tree (DT) Classifier:
Accuracy: 0.6008230452674898
Classification Report:
               precision    recall  f1-score   support

           0       0.19      0.21      0.20       195
           1       0.75      0.75      0.75       905
           2       0.10      0.08      0.09       115

    accuracy                           0.60      1215
   macro avg       0.35      0.35      0.35      1215
weighted avg       0.60      0.60      0.60      1215


Linear Support Vector Machine (SVM) Classifier:
Accuracy: 0.6

In [ ]:
print(len(loaded_bert_embeddings))
print(len(loaded_y_encoded))

In [ ]:
loaded_bert_embeddings

In [ ]:
loaded_y_encoded

In [ ]:
# end end end  end end end  end end end  end end end  end end end  end end end  end end end  end end end
# end end end  end end end  end end end  end end end  end end end  end end end  end end end  end end end
# end end end  end end end  end end end  end end end  end end end  end end end  end end end  end end end
# end end end  end end end  end end end  end end end  end end end  end end end  end end end  end end end